In [41]:
from sklearn.neighbors import KNeighborsClassifier
import warnings
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler

wight1=0
wight2=0

warnings.filterwarnings('ignore')
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        # print('-----------')
        # print(equivalence_class)
        for key_decision in equivalence_classes_dec.keys():
            # print(key_decision)
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(x)
            if(len(x)>total_records*0):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    # print(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    # if total_rule/total_records<0.8:
    #     return (999,);

    avg_confidence = np.sum(rule_confidences) / count +wight1*(entropy_D-entropy_DB)+wight2*(count/total_records)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    #random.seed(6)
    # create individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof
    # return hof

In [42]:
def bestIndividual_KNN(hof,df):
    k = 4
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('data/Zoo.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features

In [43]:
from sklearn.model_selection import StratifiedKFold
def bestIndividual_SVM(hof, df):
    def SVM(trainX, trainY, testX, testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.1, max_iter=500)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY) / testY.shape[0]
        return precision

    maxaccuracy = 0.0
    for ind in hof:
        df = pd.read_csv('data/Zoo.csv')
        feature_columns = [col for col in df.columns if col not in ['id', 'target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y = df["target"]
        X = df[selected_features]

        # Use stratified k-fold cross-validation
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        for train_index, test_index in skf.split(X, Y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index]
            trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
            score = SVM(trainX, trainY, testX, testY)
            scores.append(score)
        avg_score = np.mean(scores)

        if avg_score > maxaccuracy:
            maxaccuracy = avg_score
            best_feature = selected_features

    return maxaccuracy, best_feature

In [44]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

def bestIndividual_DT(hof, df):
    for ind in hof:
        df = pd.read_csv('data/Zoo.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        X = df[selected_features]
        Y = df['target']
        dt = DecisionTreeClassifier()
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            dt.fit(X_train, y_train)

            y_pred = dt.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)

        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    return mean_accuracy

In [45]:
wighta=[0,0.1,0.2,0.3,0.4,0.5]
wightb=[0,1,2,3,4,5]
SVM=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
KNN=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
DT=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
F=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]


for i_index, i in enumerate(wighta):
    for j_index, j in enumerate(wightb):
        try:
            # 你的代码
            wight1=i
            wight2=j
            n_pop=50
            n_gen=15
            # read dataframe from csv
            df = pd.read_csv('data/Zoo.csv')
            # encode labels column to numbers
            y = df['target']
            feature_columns = [ col for col in df.columns if col not in ['id','target']]
            X = df[feature_columns]
            # for col in feature_columns:
            #     df[col] = pd.qcut(df[col], q=5, labels=False, duplicates='drop')
            # get accuracy with all features
            individual = [1 for i in range(len(X.columns))]
            getFitness(df,'target',individual)
            print("Accuracy with all features: \t" +
                  str(getFitness(df,'target',individual)) + "\n")

            hof = geneticAlgorithm(df,'target',n_pop, n_gen)

            print('------------wight1--------------')
            print(wight1)
            print('------------wight2--------------')
            print(wight2)

            accuracy_SVM,individual = bestIndividual_SVM(hof,df)
            SVM[i_index][j_index] = accuracy_SVM
            print("-----------SVM---------------")
            print(accuracy_SVM, "保存至 SVM[{}][{}]".format(i_index, j_index))

            accuracy_KNN,individual = bestIndividual_KNN(hof,df)
            KNN[i_index][j_index] = accuracy_KNN
            print("-----------KNN---------------")
            print(accuracy_KNN, "保存至 KNN[{}][{}]".format(i_index, j_index))

            accuracy_DT = bestIndividual_DT(hof,df)
            DT[i_index][j_index] = accuracy_DT
            print("-----------DT---------------")
            print(accuracy_DT, "保存至 DT[{}][{}]".format(i_index, j_index))

            print('------------特征-----------------')
            print(individual)
            F[i_index][j_index] = individual
            print("特征保存至 F[{}][{}]".format(i_index, j_index))

        except Exception as e:
            print("执行过程出现错误：", e)
            continue

Accuracy with all features: 	(0.0,)

gen	nevals	avg     	min
0  	50    	0.152749	0  
1  	29    	0.0552385	0  
2  	32    	0.0239907	0  
3  	24    	0.00532226	0  
4  	30    	0.00572053	0  
5  	27    	0.00140909	0  
6  	34    	0.00186221	0  
7  	30    	0.000649095	0  
8  	24    	0.00305019 	0  
9  	22    	0.00169231 	0  
10 	31    	0.00144751 	0  
11 	26    	0.00252508 	0  
12 	31    	0.00148548 	0  
13 	34    	0.00108108 	0  
14 	33    	0.000418301	0  
15 	24    	0.00281305 	0  
Top 1 solution: [0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0]
Fitness: 0.0

------------wight1--------------
0
------------wight2--------------
0
-----------SVM---------------
0.9409090909090908 保存至 SVM[0][0]
-----------KNN---------------
0.8800000000000001 保存至 KNN[0][0]
-----------DT---------------
0.96 保存至 DT[0][0]
------------特征-----------------
['feathers', 'eggs', 'aquatic', 'toothed', 'backbone', 'breathes', 'venomous', 'fins', 'legs', 'domestic']
特征保存至 F[0][0]
Accuracy with all features: 	(0.58415841584

In [46]:
print(SVM)
print(KNN)
print(DT)
print(F)


[[0.9409090909090908, 0.9800000000000001, 0.930909090909091, 0.8318181818181818, 0.8318181818181818, 0.8318181818181818], [0.9400000000000001, 0.96, 0.95, 0.95, 0.8318181818181818, 0.8318181818181818], [0.9309090909090909, 0.95, 0.96, 0.8809090909090909, 0.8318181818181818, 0.8809090909090909], [0.9400000000000001, 0.9800000000000001, 0.8418181818181818, 0.8318181818181818, 0, 0.5045454545454545], [0.6227272727272727, 0.96, 0.4954545454545454, 0, 0.6036363636363635, 0], [0.5136363636363637, 0, 0, 0, 0, 0]]
[[0.8800000000000001, 0.8800000000000001, 0.890909090909091, 0.830909090909091, 0.830909090909091, 0.830909090909091], [0.9200000000000002, 0.93, 0.9200000000000002, 0.9200000000000002, 0.830909090909091, 0.830909090909091], [0.9200000000000002, 0.9200000000000002, 0.9109090909090909, 0.89, 0.830909090909091, 0.89], [0.8809090909090909, 0.9000000000000001, 0.830909090909091, 0.830909090909091, 0, 0.5018181818181818], [0.5718181818181818, 0.9400000000000001, 0.4918181818181819, 0, 0.5

In [47]:
#wighta=[0,0.1,0.2,0.3,0.4,0.5]
#wightb=[0,1,2,3,4,5]
#[[0.9509090909090909, 0.940909090909091, 0.940909090909091, 0.95, 0.8318181818181818, 0.8318181818181818], [0.9509090909090909, 0.940909090909091, 0.940909090909091, 0.8318181818181818, 0.8318181818181818, 0.8318181818181818], [0.9509090909090909, 0.940909090909091, 0.95, 0.8318181818181818, 0.8318181818181818, 0.8318181818181818], [0.9509090909090909, 0.9800000000000001, 0.95, 0.4954545454545454, 0.8318181818181818, 0.8318181818181818], [0.9209090909090909, 0.40545454545454546, 0, 0, 0, 0], [0.6427272727272727, 0, 0, 0, 0, 0]]
#[[0.89, 0.89, 0.9000000000000001, 0.9200000000000002, 0.830909090909091, 0.830909090909091], [0.89, 0.9000000000000001, 0.9000000000000001, 0.830909090909091, 0.830909090909091, 0.830909090909091], [0.89, 0.89, 0.9200000000000002, 0.830909090909091, 0.830909090909091, 0.830909090909091], [0.89, 0.9000000000000001, 0.9200000000000002, 0.4918181818181819, 0.830909090909091, 0.830909090909091], [0.8809090909090911, 0.3918181818181818, 0, 0, 0, 0], [0.6118181818181819, 0, 0, 0, 0, 0]]
#[[0.97, 0.99, 0.99, 0.95, 0.840909090909091, 0.840909090909091], [0.9700000000000001, 0.99, 0.99, 0.840909090909091, 0.840909090909091, 0.840909090909091], [0.9800000000000001, 0.99, 0.95, 0.840909090909091, 0.840909090909091, 0.840909090909091], [0.9800000000000001, 0.9800000000000001, 0.95, 0.4918181818181819, 0.840909090909091, 0.840909090909091], [0.9209090909090909, 0.40181818181818174, 0, 0, 0, 0], [0.6409090909090909, 0, 0, 0, 0, 0]]
#[[['feathers', 'eggs', 'milk', 'aquatic', 'toothed', 'backbone', 'breathes', 'venomous', 'legs'], ['eggs', 'milk', 'aquatic', 'backbone', 'fins', 'legs'], ['milk', 'aquatic', 'backbone', 'fins', 'legs'], ['feathers', 'eggs', 'milk', 'aquatic', 'backbone', 'fins'], ['feathers', 'milk', 'backbone', 'fins'], ['feathers', 'milk', 'backbone', 'fins']], [['feathers', 'eggs', 'milk', 'aquatic', 'toothed', 'backbone', 'breathes', 'venomous', 'legs'], ['milk', 'aquatic', 'backbone', 'fins', 'legs'], ['milk', 'aquatic', 'backbone', 'fins', 'legs'], ['feathers', 'milk', 'backbone', 'fins'], ['feathers', 'milk', 'backbone', 'fins'], ['feathers', 'milk', 'backbone', 'fins']], [['feathers', 'eggs', 'milk', 'aquatic', 'toothed', 'backbone', 'breathes', 'venomous', 'legs'], ['eggs', 'milk', 'aquatic', 'backbone', 'fins', 'legs'], ['feathers', 'eggs', 'milk', 'aquatic', 'backbone', 'fins'], ['feathers', 'milk', 'backbone', 'fins'], ['feathers', 'milk', 'backbone', 'fins'], ['feathers', 'milk', 'backbone', 'fins']], [['feathers', 'eggs', 'milk', 'aquatic', 'toothed', 'backbone', 'breathes', 'venomous', 'legs'], ['feathers', 'milk', 'aquatic', 'backbone', 'fins', 'legs'], ['feathers', 'eggs', 'milk', 'aquatic', 'backbone', 'fins'], ['fins'], ['feathers', 'milk', 'backbone', 'fins'], ['feathers', 'milk', 'backbone', 'fins']], [['hair', 'feathers', 'airborne', 'aquatic', 'predator', 'toothed', 'venomous', 'fins', 'legs', 'tail', 'domestic'], ['domestic'], 0, 0, 0, 0], [['feathers', 'airborne', 'domestic'], 0, 0, 0, 0, 0]]
